In [117]:
import json
import time
import webbrowser
import pandas as pd
from pandas.io.json import json_normalize
from rauth import OAuth2Service
from rauth.utils import parse_utf8_qsl
import base64

In [118]:
credentials_file = open('../oauth.json')
credentials = json.load(credentials_file)
credentials_file.close()

In [119]:
service_params = {'client_id':credentials['consumer_key'],
                  'client_secret':credentials['consumer_secret'],
                  'name':'yahoo',
                  'access_token_url':'https://api.login.yahoo.com/oauth2/get_token',
                  'authorize_url':'https://api.login.yahoo.com/oauth2/request_auth',
                  'base_url':'http://fantasysports.yahooapis.com/'
                 }

In [120]:
oauth = OAuth2Service(**service_params)

In [121]:
authorize_url = oauth.get_authorize_url(client_secret=credentials['consumer_secret'], 
                                        redirect_uri='oob', 
                                        response_type='code')

In [122]:
webbrowser.open(authorize_url)

True

In [123]:
verify = input('Enter code: ')

Enter code: 83vzyjk


In [124]:
def generate_oauth2_headers():
        """Generates header for oauth2
        """
        encoded_credentials = base64.b64encode(('{0}:{1}'.format(credentials['consumer_key'],
                                                                 credentials['consumer_secret'])).encode('utf-8'))
        headers={
            'Authorization':'Basic {0}'.format(encoded_credentials.decode('utf-8')),
            'Content-Type': 'application/x-www-form-urlencoded'
        }

        return headers

In [125]:
def oauth2_access_parser(raw_access):
        """Parse oauth2 access
        """
        parsed_access = json.loads(raw_access.content.decode('utf-8'))
        access_token = parsed_access['access_token']
        token_type = parsed_access['token_type']
        refresh_token = parsed_access['refresh_token']
        guid = parsed_access['xoauth_yahoo_guid']

        keys = {
            'access_token': access_token,
            'token_type': token_type,
            'refresh_token': refresh_token,
            'guid': guid
        }
        
        return keys
        


In [126]:
token_time = time.time()
    
keys = {'token_time': token_time}

headers = generate_oauth2_headers()
# Getting access token
raw_access = oauth.get_raw_access_token(data={"code": verify, 'redirect_uri': 'oob','grant_type':'authorization_code'}, headers=headers)
#parsed_access = parse_utf8_qsl(raw_access.content.decode('utf-8'))
keys.update(oauth2_access_parser(raw_access))

keys.update(service_params)

In [127]:
with open('secrets.json', 'w') as fp:
    json.dump(keys, fp, indent=4, sort_keys=True, ensure_ascii=False)

In [128]:
s = oauth.get_session(token=keys['access_token'])

### Get league settings and user team

In [129]:
url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nba.l.171968/settings"
params = {'format': 'json'}
response = s.get(url, params=params)

In [130]:
response_content = response.json()
num_teams = response_content['fantasy_content']['league'][0]['num_teams']

In [84]:
df_positions = pd.DataFrame(list(map(lambda x: x['roster_position'], 
    response_content['fantasy_content']['league'][1]['settings'][0]['roster_positions'])))

In [92]:
stats = [i['stat']['name']\
 for i in response_content['fantasy_content']['league'][1]['settings'][0]['stat_categories']['stats'] \
 if 'is_only_display_stat' not in i['stat']['stat_position_types'][0]['stat_position_type'].keys()]

In [93]:
stats

['Field Goal Percentage',
 'Free Throw Percentage',
 '3-point Percentage',
 'Points Scored',
 'Total Rebounds',
 'Assists',
 'Steals',
 'Blocked Shots',
 'Turnovers']

### Team Rosters

In [136]:
url = "https://fantasysports.yahooapis.com/fantasy/v2/team/nba.l.171968.t.1/roster/"
params = {'format': 'json'}
response = s.get(url, params=params)

In [137]:
response.json()

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/team/nba.l.171968.t.1/roster/',
  'team': [[{'team_key': '385.l.171968.t.1'},
    {'team_id': '1'},
    {'name': 'Klaw & Order'},
    [],
    {'url': 'https://basketball.fantasysports.yahoo.com/nba/171968/1'},
    {'team_logos': [{'team_logo': {'size': 'large',
        'url': 'https://s.yimg.com/cv/apiv2/default/nba/nba_4.png'}}]},
    [],
    {'waiver_priority': 10},
    [],
    {'number_of_moves': '16'},
    {'number_of_trades': 0},
    {'roster_adds': {'coverage_type': 'week',
      'coverage_value': 14,
      'value': '1'}},
    [],
    {'league_scoring_type': 'head'},
    [],
    {'draft_position': 9},
    {'has_draft_grade': 0},
    [],
    [],
    {'managers': [{'manager': {'manager_id': '1',
        'nickname': 'Pranav',
        'guid': 'ZDR2VHSULROLCLDGEEA4C7KB6I',
        'is_commissioner': '1',
        'email': 'pranavbadami@yahoo.com',
        'image_url': 'https://ct.yimg.com/cy/1768/39361574426_98028a_6

### Matchup Schedule

In [138]:
url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nba.l.171968/scoreboard"
params = {'format': 'json'}
response = s.get(url, params=params)

In [141]:
response.json()

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/league/nba.l.171968/scoreboard',
  'league': [{'league_key': '385.l.171968',
    'league_id': '171968',
    'name': 'This League is So Good (x4)',
    'url': 'https://basketball.fantasysports.yahoo.com/nba/171968',
    'logo_url': False,
    'draft_status': 'postdraft',
    'num_teams': 12,
    'edit_key': '2019-01-16',
    'weekly_deadline': 'intraday',
    'league_update_timestamp': '1547624692',
    'scoring_type': 'head',
    'league_type': 'private',
    'renew': '',
    'renewed': '',
    'iris_group_chat_id': 'URWADVXS2BEWFPWVLNNCDPJP44',
    'short_invitation_url': 'https://basketball.fantasysports.yahoo.com/nba/171968/invitation?key=3571f169bc17055e&ikey=15f8f95e9be7080f',
    'allow_add_to_dl_extra_pos': 1,
    'is_pro_league': '0',
    'is_cash_league': '0',
    'current_week': 14,
    'start_week': '2',
    'start_date': '2018-10-22',
    'end_week': '24',
    'end_date': '2019-04-10',
    'game_code': 'n

### Get NBA Schedule

In [115]:
import requests
url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2018/league/00_full_schedule_week.json"
response = requests.get(url).json()

In [116]:
response

{'lscd': [{'mscd': {'mon': 'September',
    'g': [{'gid': '0011800001',
      'gcode': '20180928/MELPHI',
      'seri': '',
      'is': 1,
      'gdte': '2018-09-28',
      'htm': '2018-09-28T19:00:00',
      'vtm': '2018-09-29T09:00:00',
      'etm': '2018-09-28T19:00:00',
      'an': 'Wells Fargo Center',
      'ac': 'Philadelphia',
      'as': 'PA',
      'st': '3',
      'stt': 'Final',
      'bd': {'b': [{'seq': 1,
         'disp': 'NBC Sports Philadelphia Plus',
         'scope': 'home',
         'type': 'tv',
         'lan': 'English'}]},
      'v': {'tid': 15016,
       're': '0-1',
       'ta': 'MEL',
       'tn': 'United',
       'tc': 'Melbourne',
       's': '84'},
      'h': {'tid': 1610612755,
       're': '1-0',
       'ta': 'PHI',
       'tn': '76ers',
       'tc': 'Philadelphia',
       's': '104'},
      'gweek': None,
      'gdtutc': '2018-09-28',
      'utctm': '23:00',
      'ppdst': 'I',
      'ptsls': {'pl': [{'pid': '203954',
         'fn': 'Joel',
         'ln'

In [144]:
import os
os.listdir('../data')

['player_gamelog.csv', 'player_seasons.csv']

### Player gamelogs

In [189]:
df_gamelog = pd.read_csv('../data/player_gamelog.csv').drop_duplicates(subset=['PLAYER_ID', 'GAME_ID'])

In [162]:
df_gamelog.iloc[:5, 10:20]

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT
0,22.9,4.0,11.0,0.364,0.0,0.0,0.0,0.0,0.0,0.000
1,37.0,11.0,17.0,0.647,0.0,0.0,0.0,6.0,6.0,1.000
2,32.8,8.0,14.0,0.571,0.0,1.0,0.0,1.0,1.0,1.000
3,48.6,20.0,33.0,0.606,0.0,0.0,0.0,16.0,16.0,1.000
4,29.6,4.0,12.0,0.333,0.0,0.0,0.0,5.0,6.0,0.833


In [196]:
pct_dict

{'2017-18': {'FGM': 3.7321408051480445,
  'FGA': 8.109204427931207,
  'FG3M': 0.9885088290496802,
  'FG3A': 2.7325621480828897,
  'FTM': 1.5667445512697744,
  'FTA': 2.042555636419351},
 '2018-19': {'FGM': 3.8276223776223777,
  'FGA': 8.323776223776223,
  'FG3M': 1.0395104895104894,
  'FG3A': 2.9302797202797204,
  'FTM': 1.6646853146853147,
  'FTA': 2.1793706293706294}}

In [199]:
pct_dict = df_gamelog[['SEASON_YEAR', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA']]\
.groupby('SEASON_YEAR').mean().to_dict(orient='index')

stats = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'PTS', 'AST', 'REB', 'BLK', 'STL', 'TOV']
def aggregate_stats(df, pct_dict, stats, n_players):
    season_dict = pct_dict[df.SEASON_YEAR.iloc[0]]
    agg_dict = {k:'mean' for k in stats}
    agg_dict['PLAYER_NAME'] = 'size'
    df_agg = df[['PLAYER_ID', 'PLAYER_NAME'] + stats].groupby(['PLAYER_ID', 'PLAYER_NAME']).agg(agg_dict)
    
    df_agg['TOV'] = -df_agg['TOV']

    df_agg['FG_PCT'] = (df_agg['FGM'] + season_dict['FGM'] * (n_players - 1))/ \
                        (df_agg['FGA'] + season_dict['FGA']* (n_players - 1))
    df_agg['FG3_PCT'] = (df_agg['FG3M'] + season_dict['FG3M'] * (n_players - 1))/ \
                         (df_agg['FG3A'] + season_dict['FG3A']* (n_players - 1))
    df_agg['FT_PCT'] = (df_agg['FTM'] + season_dict['FTM'] * (n_players - 1))/ \
                        (df_agg['FTA'] + season_dict['FTA'] * (n_players - 1))
    
    final_stats = ['PTS', 'AST', 'BLK', 'REB', 'STL', 'TOV', 'FG_PCT', 'FG3_PCT', 'FT_PCT']
    
    df_agg[final_stats] = df_agg[final_stats].apply(lambda x: (x - x.min())/(x.max() - x.min()))
    df_agg.rename(columns = {'PLAYER_NAME':'GP'}, inplace=True)
    
    df_agg = df_agg[final_stats + ['GP']]
    df_agg['AVG_SCORE'] = df_agg[final_stats].sum(axis=1)
    df_agg['TOTAL_SCORE'] = df_agg['AVG_SCORE'] * df_agg['GP']
    
    return df_agg
    

In [202]:
df = df_gamelog.groupby('SEASON_YEAR').apply(lambda x: aggregate_stats(x, pct_dict, stats, 13)).reset_index()

In [203]:
df[df.SEASON_YEAR == '2018-19'].sort_values('AVG_SCORE', ascending=False).head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,PTS,AST,BLK,REB,STL,TOV,FG_PCT,FG3_PCT,FT_PCT,GP,AVG_SCORE,TOTAL_SCORE
684,2018-19,203076,Anthony Davis,0.831310,0.414844,0.938889,0.888402,0.692308,0.635855,0.733009,0.542748,0.734986,40,6.412351,256.494050
581,2018-19,201142,Kevin Durant,0.798008,0.566667,0.385185,0.490254,0.325275,0.412573,0.718175,0.646360,1.000000,45,5.342497,240.412369
667,2018-19,202695,Kawhi Leonard,0.781513,0.294271,0.190586,0.522913,0.769231,0.635356,0.700700,0.606151,0.832070,36,5.332792,191.980498
619,2018-19,201939,Stephen Curry,0.843286,0.510110,0.127451,0.343860,0.488688,0.460526,0.653535,1.000000,0.901985,34,5.329441,181.200999
616,2018-19,201935,James Harden,1.000000,0.800305,0.228997,0.418868,0.800858,0.000000,0.403496,0.667208,0.978464,41,5.298196,217.226033
